# Libraries

In [1]:
import gym
import time
import matplotlib.pyplot as plt
from pyswip import Prolog
from utils import process_state, perform_action, show_match
from nle import nethack
from run import completemapdes,roomroomdes,mazeroomdes
import random
from utilsbattle import get_player_location, get_target_location,get_money_location,get_closest_gold, chebyshev_distance,actions_from_path,get_stair_location
from algorithms import a_star
import IPython.display as display
import copy as cp

ModuleNotFoundError: No module named 'algorithms'

# Simulation parameters

In [ ]:
NUM_EPISODES = 5
MAX_STEPS = 100
PATH = 'kb-battle.pl'
RISK_THRESHOLD=1# when the agent feels it's not healthy anymore
MONEY_THRESHOLD=1
MONEY=40
ARROWS= 2
ARROWS_THRESHOLD=4

Initialize the knowledge base.

In [ ]:
KB = Prolog()
KB.consult(PATH)

KB.retractall('onPlan(_)')
#caricamento fatti nella KB
#KB.asserta(f"health_threshold({RISK_THRESHOLD})")
#KB.asserta(f"money_threshold({MONEY_THRESHOLD})")
KB.asserta(f"money({MONEY})")
#KB.asserta(f"arrows({ARROWS})")
#KB.asserta(f"arrows_threshold({ARROWS_THRESHOLD})")
#KB.asserta(f"corridors")



#### Main code
- Perform `NUM_EPISODES` experiences in the environment.
- Use `Prolog` to define the axioms and choose the action to perform.
- The main goal is to _reach and eat_ the `apple`.

In [ ]:
rewards = []
action=None
for episode in range(NUM_EPISODES):
    des_file= completemapdes(1,20)  #creashop(episode+1,random.randint(0,200))
    actions=tuple(nethack.CompassDirection )+(nethack.Command.PICKUP,nethack.Command.PAY,nethack.Command.WIELD)

    env = gym.make('MiniHack-Skill-Custom-v0',
               observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
               des_file=des_file)

    # count the number of steps of the current episode
    steps = 0
    # store the cumulative reward
    reward = 0.0
    # collect obs['pixel'] to visualize
    ep_states = []

    obs = env.reset()
    env.render()
    plt.imshow(obs['pixel'][0:1000, 200:1000])
    ep_states.append(obs['pixel'])
    done = False

    # Main loop
    while not done and steps < MAX_STEPS:
        # Get the observation from the env and assert the facts in the kb 
        process_state(obs, KB)
        try:
            actions=list(KB.query('action(X)'))
            action = actions[0]
            action = action['X']

        except Exception as e:
            action = None
        # Perform the action in the environment
        if action:
            obs, reward, done, info = perform_action(action, env,KB)
            ep_states.append(obs['pixel'])
            env.render()
        else:
            env.render()
            break
        steps += 1
    # Display game with interface
    #show_match(ep_states)
    
    rewards.append(reward)
    # reset the environment and retract axioms that may cause errors
    obs = env.reset()
    KB.retractall('stepping_on(agent,_,_)')
    KB.retractall('position(_,_,_,_)')


print(f'After {NUM_EPISODES} episodes, mean return is {sum(rewards)/NUM_EPISODES}')
print("The rewards of the episodes are:", rewards)

-------------------------------------

In [ ]:
des_file= roomroomdes(20,1)
env = gym.make('MiniHack-Skill-Custom-v0',
               observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
               des_file=des_file)
state = env.reset()
env.render()
        



In [ ]:
statuses = list()
debug_kb = None
for episode in range(100):
    print(f'Episode: {episode} Start')
    des_file= mazeroomdes(4)
    env = gym.make('MiniHack-Skill-Custom-v0', 
                    observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
                    max_episode_steps = MAX_STEPS,
                    des_file=des_file)
    obs = env.reset()

    game_map = obs['chars']
    game = obs['pixel']

    #retrieve map informations
    start = get_player_location(game_map)
    target = get_target_location(game_map)
    stair_des_pos = get_stair_location(des_file)
    delta_x, delta_y = (target[0]-stair_des_pos[0],target[1]-stair_des_pos[1])
    gold_pos_list = get_money_location(des_file) #attento a formato tuple in lista

    gold_pos_list = [(x+delta_x,y+delta_y)for (_,(x,y)) in gold_pos_list]

    #put gold positions in kb

    for (x,y) in gold_pos_list :
        KB.asserta(f'position(gold,_,{x},{y})')

    KB.asserta(f'position(stairs,_,{target[0]},{target[1]})')


    steps = 0
    done = False
    planned_actions = None
    ep_states = list()
    ep_states.append(obs['pixel'])

    #print(list(KB.query('position(gold,_,_,_)')))

    while not done and steps < MAX_STEPS:

        #print('Prima del process')
        #print(list(KB.query('position(gold,_,_,_)')))

        #update Kb with current information
        process_state(obs,KB)

        #choose next action
        try:
            #print(f'Step: {steps}')
            actions=list(KB.query('action(X)'))
            #print(actions)
            action = actions[0]
            action = action['X']
            #print(f'>> Current action from Prolog: {action}')
        except Exception as e:
            print(e)
            action = None
        # Perform the action in the environment
        if action:
            try :
                obs, reward, done, info, planned_actions = perform_action(action, env,KB,planned_actions,obs)
                ep_states.append(obs['pixel'])
            except Exception as e :
                print(e)
                debug_kb = cp.deepcopy(KB)
                raise Exception(e)
                break
            #env.render()
        else:
            #env.render()
            break
        steps += 1

    #show_match(ep_states)
    

    statuses.append(info['end_status'])
    obs = env.reset()
    KB.retractall('stepping_on(agent,_,_)')
    KB.retractall('position(_,_,_,_)')
    KB.retractall('onPlan(_)')
    KB.retractall('money(_)')
    KB.asserta(f"money({MONEY})")
        
        

Episode: 0 Start
Episode: 1 Start
Episode: 2 Start
Episode: 3 Start
Episode: 4 Start
Episode: 5 Start
Episode: 6 Start
Episode: 7 Start
Episode: 8 Start
Episode: 9 Start
Episode: 10 Start
Episode: 11 Start
Episode: 12 Start
Episode: 13 Start
Episode: 14 Start
Episode: 15 Start
Episode: 16 Start
Episode: 17 Start
Episode: 18 Start
Episode: 19 Start
Episode: 20 Start
Episode: 21 Start
Episode: 22 Start
Episode: 23 Start
Episode: 24 Start
Episode: 25 Start
Episode: 26 Start
Episode: 27 Start
Episode: 28 Start
Episode: 29 Start
Episode: 30 Start
Episode: 31 Start
Episode: 32 Start
Episode: 33 Start
Episode: 34 Start
Episode: 35 Start
Episode: 36 Start
Episode: 37 Start
Episode: 38 Start
Episode: 39 Start
Episode: 40 Start
Episode: 41 Start
Episode: 42 Start
Episode: 43 Start
Episode: 44 Start
Episode: 45 Start
Episode: 46 Start
Episode: 47 Start
Episode: 48 Start
Episode: 49 Start
Episode: 50 Start
Episode: 51 Start
Episode: 52 Start
Episode: 53 Start
cannot unpack non-iterable NoneType ob

Exception: cannot unpack non-iterable NoneType object

In [ ]:
des_file= mazeroomdes(4)
env = gym.make('MiniHack-Skill-Custom-v0', 
                    observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
                    max_episode_steps = MAX_STEPS,
                    des_file=des_file)
obs = env.reset()
env.render()

In [ ]:
len([1 for el in statuses if el == statuses[0]])

52